In [ ]:
import gurobipy as grb
from gurobipy import *
import numpy as np

In [ ]:
# Create a new model
PRA = Model("PRA")

# Create variables
num_pairs = 297
num_links = 15

x = PRA.addVars(['x_' + str(i) for i in range(num_pairs)], lb=0, obj=1, vtype=GRB.CONTINUOUS)
y = PRA.addVars(['y_' + str(i) for i in range(num_links)],lb=0, obj=1, vtype=GRB.CONTINUOUS)
z = PRA.addVars(['z_' + str(i) for i in range(num_pairs)],lb=0, obj=1, vtype=GRB.CONTINUOUS)
t = PRA.addVars(['t_' + str(i) for i in range(num_pairs)],lb=0, ub=1, obj=1, vtype=GRB.CONTINUOUS)

# Integrate new variables
PRA.update()

In [ ]:
prior_od = 5 + 5 * np.random.rand(num_pairs)
prior_link = 10 + 10 * np.random.rand(num_links)
prior_pr = 1 + np.random.rand(num_pairs)

obj_term_od = x.values() - prior_od
obj_term_od *= obj_term_od
obj_term_link = y.values() - prior_link
obj_term_link *= obj_term_link
obj_term_pr = z.values() - prior_pr
obj_term_pr *= obj_term_pr


obj_term = 0
for term in obj_term_od:
    obj_term += term
for term in obj_term_link:
    obj_term += term
for term in obj_term_pr:
    obj_term += term

# Set objective
PRA.setObjective(obj_term, GRB.MINIMIZE)

In [ ]:
# Add constraints
PRA.params.NonConvex = 2


# Constraint #1: traffic assignment
incidence_matrix = np.round(np.random.rand(len(x),len(y)))
assignment_fraction = np.random.rand(len(x),len(y))
for idx, yy in enumerate(y.values()):
    rhs = x.values() * incidence_matrix[:,idx] * assignment_fraction[:,idx]
    rhs_sum = 0
    for term in rhs:
        rhs_sum += term
    PRA.addConstr(yy == rhs_sum)


# Constraint #2: Penetration rate assignment
for idx, xx in enumerate(x.values()):
    upside = np.random.rand()
    PRA.addConstr(t.values()[idx] * xx - upside <= 1)
    PRA.addConstr(t.values()[idx] * xx - upside >= -1)

incidence_matrix_pr = np.round(np.random.rand(len(t),len(z)))
assignment_fraction_pr = np.random.rand(len(t),len(z))
for idx, zz in enumerate(z.values()):
    rhs = t.values() * incidence_matrix_pr[:,idx] * assignment_fraction_pr[:,idx]
    rhs_sum = 0
    for term in rhs:
        rhs_sum += term
    PRA.addConstr(zz - rhs_sum <= 0.5)
    PRA.addConstr(zz - rhs_sum >= -0.5)

In [ ]:
PRA.optimize()

In [ ]:
solution_x = [v.x for v in PRA.getVars()[:num_pairs]]
solution_y = [v.x for v in PRA.getVars()[num_pairs:(num_links + num_pairs)]]
solution_z = [v.x for v in PRA.getVars()[(num_links + num_pairs):(num_links + 2 * num_pairs)]]